In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install cartopy rasterio rasterstats -q

In [ ]:
import ee
import geopandas as gpd
import pandas as pd
import geopandas as gpd
import numpy as np
import time
from rasterstats import zonal_stats
import rasterio
import os
import cartopy.crs as ccrs

import glob
import requests
import zipfile
from osgeo import gdal, osr, ogr
import numpy as np
import subprocess
from rasterio.warp import reproject, Resampling
from rasterio.windows import Window
from concurrent.futures import ThreadPoolExecutor


In [ ]:
base_folder = '/content/drive/MyDrive/hwi_product'
output_folder = '/content/drive/MyDrive/hwi_stats/dgca_back'

In [ ]:

# Define the input files for each decade
decades = ['1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s']
data_files = {decade: os.path.join(output_folder, f'average_hwi_{decade}.tif') for decade in decades}

# Initialize dictionary to hold data for each decade
data_decades = {decade: {} for decade in decades}

with rasterio.open('/content/drive/MyDrive/hwi_product/hwi_1960s/temp.vrt') as src:
    # Read the land mask
    land_mask = src.read(1)
    land_mask_nan = np.where(land_mask != 1, np.nan, 1)

# Function to read data from a file and store in the dictionary
def read_data(file_path, data_dict):
    if os.path.exists(file_path):
        with rasterio.open(file_path) as src:
            data_dict['hw_count'] = src.read(1) * land_mask_nan
            data_dict['hw_days'] = src.read(2)/src.read(1) * land_mask_nan
            data_dict['hw_temp_diff'] = src.read(3) * land_mask_nan
            data_dict['high_temp_degree_days'] = src.read(4) * land_mask_nan

# Read data for each decade
for decade in decades:
    read_data(data_files[decade], data_decades[decade])

# Combine data for all decades to calculate overall mean and standard deviation
combined_data = {indicator: [] for indicator in data_decades['1960s'].keys()}
for decade in decades:
    for indicator in combined_data.keys():
        combined_data[indicator].append(data_decades[decade][indicator])

# Calculate global mean and standard deviation across all decades and spatial dimensions
mean_values = {indicator: np.nanmean(np.stack(combined_data[indicator]), axis=(0, 1, 2)) for indicator in combined_data.keys()}
std_dev = {indicator: np.nanstd(np.stack(combined_data[indicator]), axis=(0, 1, 2)) for indicator in combined_data.keys()}



In [ ]:
df_wpp_prediction = pd.read_csv("/content/drive/MyDrive/CCRI/WPP2022_prediction.csv")
df_wpp_estimate = pd.read_csv("/content/drive/MyDrive/CCRI/WPP2022_estimate.csv")
df_wpp = pd.concat([df_wpp_estimate,df_wpp_prediction], ignore_index=True)
country_bnd = gpd.read_file('/content/drive/MyDrive/CCRI/global_bnd_adm0.geojson')

# Merge polygons with the same ISO3 code
country_bnd = country_bnd.dissolve(by='iso3')
country_bnd = country_bnd.reset_index()


In [ ]:
import os
import rasterio
import numpy as np
import pandas as pd
from shapely.geometry import mapping
from rasterio.mask import mask
from concurrent.futures import ProcessPoolExecutor

def calculate_exposure(iso3, pop_vrt_path, file_T, df_wpp, year, output_dir, mean_values, std_dev):
    """Calculates exposure metrics for a given ISO3 code and writes results to a CSV file."""
    print(f"Processing {iso3} , {year}")

    child_percent = df_wpp.loc[(df_wpp['ISO3 Alpha-code'] == iso3) & (df_wpp['Year'] == year), '0-17']
    if child_percent.empty:
      print(f"No population data for {iso3} in {year}")
      return None

    filtered_gdf = country_bnd[country_bnd['iso3'] == iso3]
    # Create a temporary filtered GeoJSON file
    filtered_geojson_path = f'/content/drive/MyDrive/POP_stat_extreme/filtered_{iso3}.geojson'
    filtered_gdf.to_file(filtered_geojson_path, driver='GeoJSON')

    subset_pop_path = os.path.join(output_dir, f"{iso3}_pop_subset.tif")
    subset_T_path = os.path.join(output_dir, f"{iso3}_T_subset.tif")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    def clip_raster(input_path, output_path, filtered_geojson_path, iso3_value):

      # Clip the raster using gdalwarp with the filtered GeoJSON file
      warp_options = gdal.WarpOptions(
          cutlineDSName=filtered_geojson_path,
          cropToCutline=True,
          dstAlpha=True,
          format='GTiff',
          creationOptions=["COMPRESS=LZW"]
      )

      gdal.Warp(destNameOrDestDS=output_path, srcDSOrSrcDSTab=input_path, options=warp_options)

    # Clip each input raster to the subset paths
    try:
        clip_raster(pop_vrt_path, subset_pop_path, filtered_geojson_path, iso3)
        clip_raster(file_T, subset_T_path, filtered_geojson_path, iso3)
    except Exception as e:
        print(f"Error during raster clipping: {e}")
        return None

    data_T = {}
    pop_band = (year - 1975) // 5 + 1
    try:
        with rasterio.open(subset_pop_path) as pop_src:
            total_pop = np.nansum(pop_src.read(pop_band))
            child_percent = df_wpp.loc[(df_wpp['ISO3 Alpha-code'] == iso3) & (df_wpp['Year'] == year), '0-17']
            if child_percent.empty:
                print(f"No population data for {iso3} in {year}")
                for path in [subset_pop_path, subset_T_path]:
                    os.remove(path)
                return None
            child_percent = float(child_percent.values[0])
            child_pop = pop_src.read(pop_band) * (child_percent / 100)
    except Exception as e:
        print(f"Error reading {subset_pop_path}: {e}")
        for path in [subset_pop_path, subset_T_path]:
            os.remove(path)
        return None

    def read_data(file_path, data_dict):
        try:
            with rasterio.open(file_path) as src:
                data_dict['hw_count'] = src.read(1)
                with np.errstate(divide='ignore', invalid='ignore'):
                    hw_count = src.read(1)
                    hw_days = src.read(2)
                    hw_days_per_count = np.where(hw_count == 0, 0, hw_days / hw_count)
                    data_dict['hw_days'] = hw_days_per_count
                data_dict['hw_temp_diff'] = src.read(3)
                data_dict['high_temp_degree_days'] = src.read(4)
        except Exception as e:
            print(f"Error reading {file_path}: {e}")

    read_data(subset_T_path, data_T)

    results = {'iso3': iso3}
    results["total_pop"] = total_pop
    results["child_pop"] = np.nansum(child_pop)
    results["child_percent"] = child_percent

    for key in data_T.keys():
        for std in [1, 2, 3]:
            threshold_value = mean_values[key] + std * std_dev[key]
            mask_high_percentage = data_T[key] > threshold_value
            results[f"exposure_{key}_{std}std_{year-5}s"] = np.nansum(child_pop[mask_high_percentage])

    # Clean up temporary subset files (optional)
    for path in [subset_pop_path, subset_T_path, filtered_geojson_path]:
        os.remove(path)

    return results


In [ ]:

# Path to the VRT file
pop_vrt_path = "/content/drive/MyDrive/GHS_POP_DATA/GHS_POP.vrt"

# Create the output directory if it doesn't exist
output_dir = "/content/drive/MyDrive/POP_stat_extreme"
os.makedirs(output_dir, exist_ok=True)

for year in range(1970, 2030, 10):
  file_T = f"/content/drive/MyDrive/hwi_stats/dgca/average_hwi_{year}s.tif"
  pop_year = year +5

  results_list = []
  iso3_codes = country_bnd['iso3'].unique()

  with ThreadPoolExecutor(max_workers=3) as executor:
      futures = {executor.submit(calculate_exposure, iso3, pop_vrt_path, file_T, df_wpp, pop_year, output_dir, mean_values, std_dev): iso3 for iso3 in iso3_codes}
      for future in futures:
          result = future.result()
          if result is not None:
              results_list.append(result)

  # Combine results and write to a single CSV
  all_results_df = pd.DataFrame(results_list)
  all_results_df.to_csv(os.path.join(output_dir, f"extreme_exposure_results_{year}s_high_res.csv"), index=False)